# GAMMA 挑战赛任务一

赛题链接

    MICCAI2021 Contest - GAMMA: https://aistudio.baidu.com/aistudio/competition/detail/90

比赛简介

    GAMMA挑战赛是由百度在MICCAI2021研讨会OMIA8上举办的国际眼科赛事。MICCAI是由国际医学图像计算和计算机辅助干预协会举办的跨医学影像计算和计算机辅助介入两个领域的综合性学术会议，是该领域的顶级会议。OMIA是百度在MICCAI会议上组织的眼科医学影像分析研讨会，至今已举办八届。
    
    本次GAMMA挑战赛聚焦多模态影像中的青光眼分析，共包括三个子任务：1）青光眼分级；2）黄斑中央凹定位；3）视杯&视盘分割。
    
任务说明

    对于GAMMA比赛任务1，目的是基于2D眼底彩照和3D OCT图像进行青光眼程度的分类。共分为3类：无青光眼、早期青光眼、中期及晚期青光眼。

数据集说明

    使用的数据集为GAMMA比赛释放的多模态眼底图像。
    
方案介绍

    本项目在基线的基础上，使用了Resnet152，经过多次训练得到了8.95的成绩。这一成绩的偶然性较高，在当前模型下可能难以获得更好的成绩了。

# 测试记录

| index | 版本 | score | Kappa | 备注 |
| -------- | -------- | -------- | -------- | -------- |
| 0     | 版本0     | 6.05162	|0.60516     | 官方基线，保存为best_model_0.7458|
|1     | 版本1     | 7.6173	|0.76173     | 全部Resnet使用152，保存为best_model_0.9123|
|2     | 版本1     | 6.84392	|0.68439     | 接续1进行测试，保存为best_model_0.8276|
|3     | 版本1     |8.09102	|0.8091     | 接续1进行测试，保存为best_model_0.8157|
|4     | 版本1     |7.97508	|0.79751     | 接续3进行测试，也许应该重新划分数据集|
|5     | 版本1     |8.18841	|0.81884     | 接续3进行测试，保存为best_model_0.9231|
|6     | 版本2     |8.52667	|0.85267	     | 接续5进行测试，保存为best_model_8.52667|
|7     | 版本2     |8.95949	|0.89595	     | 接续6进行测试，保存为best_model_8.95|

- 解压数据包

In [1]:
! wget https://dataset-bj.cdn.bcebos.com/%E5%8C%BB%E7%96%97%E6%AF%94%E8%B5%9B/task1_gamma_grading.tar.gz.00
! wget https://dataset-bj.cdn.bcebos.com/%E5%8C%BB%E7%96%97%E6%AF%94%E8%B5%9B/task1_gamma_grading.tar.gz.01

In [2]:
! cat task1_gamma_grading.tar.gz* | tar -xzv

- 导入包

In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.vision.models import resnet34
import paddle.vision.models

import transforms as trans

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


# 配置

In [2]:
batchsize = 16 # 批大小,
oct_img_size = [512, 512]
image_size = 256
iters = 1000 # 迭代次数
val_ratio = 0.2 # 训练/验证数据划分比例，80 / 20
trainset_root = "Glaucoma_grading/training/multi-modality_images"
testset_root = "Glaucoma_grading/testing/multi-modality_images"
num_workers = 4
init_lr = 1e-4
optimizer_type = "adam"

# 训练 / 验证数据划分

In [3]:
filelists = os.listdir(trainset_root)
# train_filelists, val_filelists = train_test_split(filelists, test_size=val_ratio, random_state=42)
train_filelists, val_filelists = train_test_split(filelists, test_size=val_ratio)
print("Total Nums: {}, train: {}, val: {}".format(len(filelists), len(train_filelists), len(val_filelists)))

Total Nums: 100, train: 80, val: 20


# 数据加载

- 根据“患者id”加载oct图像和眼底图像
        

In [5]:
class GAMMA_sub1_dataset(paddle.io.Dataset):
    """
    getitem() output:
    
    	fundus_img: RGB uint8 image with shape (3, image_size, image_size)
        
        oct_img:    Uint8 image with shape (256, oct_img_size[0], oct_img_size[1])
    """

    def __init__(self,
                 img_transforms,
                 oct_transforms,
                 dataset_root,
                 label_file='',
                 filelists=None,
                 num_classes=3,
                 mode='train'):

        self.dataset_root = dataset_root
        self.img_transforms = img_transforms
        self.oct_transforms = oct_transforms
        self.mode = mode.lower()
        self.num_classes = num_classes
        
        if self.mode == 'train':
            label = {row['data']: row[1:].values 
                        for _, row in pd.read_excel(label_file).iterrows()}

            self.file_list = [[f, label[int(f)]] for f in os.listdir(dataset_root)]
        elif self.mode == "test":
            self.file_list = [[f, None] for f in os.listdir(dataset_root)]
        
        if filelists is not None:
            self.file_list = [item for item in self.file_list if item[0] in filelists]

    def __getitem__(self, idx):
        real_index, label = self.file_list[idx]

        fundus_img_path = os.path.join(self.dataset_root, real_index, real_index + ".jpg")
        oct_series_list = sorted(os.listdir(os.path.join(self.dataset_root, real_index, real_index)), 
                                    key=lambda x: int(x.strip("_")[0]))

        fundus_img = cv2.imread(fundus_img_path)[:, :, ::-1] # BGR -> RGB
        oct_series_0 = cv2.imread(os.path.join(self.dataset_root, real_index, real_index, oct_series_list[0]), 
                                    cv2.IMREAD_GRAYSCALE)
        oct_img = np.zeros((len(oct_series_list), oct_series_0.shape[0], oct_series_0.shape[1], 1), dtype="uint8")

        for k, p in enumerate(oct_series_list):
            oct_img[k] = cv2.imread(
                os.path.join(self.dataset_root, real_index, real_index, p), cv2.IMREAD_GRAYSCALE)[..., np.newaxis]

        if self.img_transforms is not None:
            fundus_img = self.img_transforms(fundus_img)
        if self.oct_transforms is not None:
            oct_img = self.oct_transforms(oct_img)
 
        # normlize on GPU to save CPU Memory and IO consuming.
        # fundus_img = (fundus_img / 255.).astype("float32")
        # oct_img = (oct_img / 255.).astype("float32")

        fundus_img = fundus_img.transpose(2, 0, 1) # H, W, C -> C, H, W
        oct_img = oct_img.squeeze(-1) # D, H, W, 1 -> D, H, W

        if self.mode == 'test':
            return fundus_img, oct_img, real_index
        if self.mode == "train":
            label = label.argmax()
            return fundus_img, oct_img, label

    def __len__(self):
        return len(self.file_list)

In [6]:
img_train_transforms = trans.Compose([
    trans.RandomResizedCrop(
        # image_size, scale=(0.90, 1.1), ratio=(0.90, 1.1)),
        image_size, scale=(0.70, 1.3), ratio=(0.70, 1.3)),
    trans.RandomHorizontalFlip(),
    trans.RandomVerticalFlip(),
    trans.RandomRotation(60)
])

oct_train_transforms = trans.Compose([
    trans.CenterCrop([256] + oct_img_size),
    trans.RandomHorizontalFlip(),
    trans.RandomVerticalFlip(),
    trans.RandomRotation(60)
])

img_val_transforms = trans.Compose([
    trans.CropCenterSquare(),
    trans.Resize((image_size, image_size))
])

oct_val_transforms = trans.Compose([
    trans.CenterCrop([256] + oct_img_size)
])

# 网络模型

In [7]:
class Model(nn.Layer):
    """
    simply create a 2-branch network, and concat global pooled feature vector.
    each branch = single resnet34
    """
    def __init__(self):
        super(Model, self).__init__()
        self.fundus_branch = paddle.vision.models.resnet152(pretrained=True, num_classes=0) # 移除最后一层全连接层
        self.oct_branch = paddle.vision.models.resnet152(pretrained=True, num_classes=0) # 移除最后一层全连接层
        self.decision_branch = nn.Linear(2048 * 1 * 2, 3) 
        
        # 在oct_branch更改第一个卷积层通道数
        self.oct_branch.conv1 = nn.Conv2D(256, 64,
                                        kernel_size=7,
                                        stride=2,
                                        padding=3,
                                        bias_attr=False)

    def forward(self, fundus_img, oct_img):
        b1 = self.fundus_branch(fundus_img)
        b2 = self.oct_branch(oct_img)
        b1 = paddle.flatten(b1, 1)
        b2 = paddle.flatten(b2, 1)
        logit = self.decision_branch(paddle.concat([b1, b2], 1))

        return logit

# 功能函数

In [8]:
def train(model, iters, train_dataloader, val_dataloader, optimizer, criterion, log_interval, eval_interval):
    iter = 0
    model.train()
    avg_loss_list = []
    avg_kappa_list = []
    best_kappa = 0.
    while iter < iters:
        for data in train_dataloader:
            iter += 1
            if iter > iters:
                break
            fundus_imgs = (data[0] / 255.).astype("float32")
            oct_imgs = (data[1] / 255.).astype("float32")
            labels = data[2].astype('int64')

            logits = model(fundus_imgs, oct_imgs)
            loss = criterion(logits, labels)
            # acc = paddle.metric.accuracy(input=logits, label=labels.reshape((-1, 1)), k=1)
            for p, l in zip(logits.numpy().argmax(1), labels.numpy()):
                avg_kappa_list.append([p, l])

            loss.backward()
            optimizer.step()

            model.clear_gradients()
            avg_loss_list.append(loss.numpy()[0])

            if iter % log_interval == 0:
                avg_loss = np.array(avg_loss_list).mean()
                avg_kappa_list = np.array(avg_kappa_list)
                avg_kappa = cohen_kappa_score(avg_kappa_list[:, 0], avg_kappa_list[:, 1], weights='quadratic')
                avg_loss_list = []
                avg_kappa_list = []
                print("[TRAIN] iter={}/{} avg_loss={:.4f} avg_kappa={:.4f}".format(iter, iters, avg_loss, avg_kappa))

            if iter % eval_interval == 0:
                avg_loss, avg_kappa = val(model, val_dataloader, criterion)
                print("[EVAL] iter={}/{} avg_loss={:.4f} kappa={:.4f}".format(iter, iters, avg_loss, avg_kappa))
                # if avg_kappa >= best_kappa:
                if 1:
                    best_kappa = avg_kappa
                    paddle.save(model.state_dict(),
                            os.path.join("{}_best_model_{:.4f}".format(iter,best_kappa), 'model.pdparams'))
                model.train()

def val(model, val_dataloader, criterion):
    model.eval()
    avg_loss_list = []
    cache = []
    with paddle.no_grad():
        for data in val_dataloader:
            fundus_imgs = (data[0] / 255.).astype("float32")
            oct_imgs = (data[1] / 255.).astype("float32")
            labels = data[2].astype('int64')
            
            logits = model(fundus_imgs, oct_imgs)
            for p, l in zip(logits.numpy().argmax(1), labels.numpy()):
                cache.append([p, l])

            loss = criterion(logits, labels)
            # acc = paddle.metric.accuracy(input=logits, label=labels.reshape((-1, 1)), k=1)
            avg_loss_list.append(loss.numpy()[0])
    cache = np.array(cache)
    kappa = cohen_kappa_score(cache[:, 0], cache[:, 1], weights='quadratic')
    avg_loss = np.array(avg_loss_list).mean()

    return avg_loss, kappa

# 训练阶段

In [9]:
img_train_transforms = trans.Compose([
    trans.RandomResizedCrop(
        image_size, scale=(0.90, 1.1), ratio=(0.90, 1.1)),
    trans.RandomHorizontalFlip(),
    trans.RandomVerticalFlip(),
    trans.RandomRotation(30)
])

oct_train_transforms = trans.Compose([
    trans.CenterCrop([256] + oct_img_size),
    trans.RandomHorizontalFlip(),
    trans.RandomVerticalFlip()
])

img_val_transforms = trans.Compose([
    trans.CropCenterSquare(),
    trans.Resize((image_size, image_size))
])

oct_val_transforms = trans.Compose([
    trans.CenterCrop([256] + oct_img_size)
])

train_dataset = GAMMA_sub1_dataset(dataset_root=trainset_root, 
                        img_transforms=img_train_transforms,
                        oct_transforms=oct_train_transforms,
                        filelists=train_filelists,
                        label_file='Glaucoma_grading/training/glaucoma_grading_training_GT.xlsx')

val_dataset = GAMMA_sub1_dataset(dataset_root=trainset_root, 
                        img_transforms=img_val_transforms,
                        oct_transforms=oct_val_transforms,
                        filelists=val_filelists,
                        label_file='Glaucoma_grading/training/glaucoma_grading_training_GT.xlsx')

In [10]:
train_loader = paddle.io.DataLoader(
    train_dataset,
    batch_sampler=paddle.io.DistributedBatchSampler(train_dataset, batch_size=batchsize, shuffle=True, drop_last=False),
    num_workers=num_workers,
    return_list=True,
    use_shared_memory=False
)

val_loader = paddle.io.DataLoader(
    val_dataset,
    batch_sampler=paddle.io.DistributedBatchSampler(val_dataset, batch_size=batchsize, shuffle=True, drop_last=False),
    num_workers=num_workers,
    return_list=True,
    use_shared_memory=False
)

In [11]:
model = Model()

if optimizer_type == "adam":
    optimizer = paddle.optimizer.Adam(init_lr, parameters=model.parameters())

criterion = nn.CrossEntropyLoss()

In [12]:
# 接续上次的运行结果
best_model_path = "./best_model_8.95/model.pdparams"
para_state_dict = paddle.load(best_model_path)
model.set_state_dict(para_state_dict)

In [13]:
train(model, iters, train_loader, val_loader, optimizer, criterion, log_interval=10, eval_interval=100)

# 预测阶段

100 8.89641
200 7.8391

In [20]:
best_model_path = "./200_best_model_1.0000/model.pdparams"
model = Model()
para_state_dict = paddle.load(best_model_path)
model.set_state_dict(para_state_dict)
model.eval()

2021-11-13 17:53:38,331 - INFO - unique_endpoints {''}
2021-11-13 17:53:38,332 - INFO - File /home/aistudio/.cache/paddle/hapi/weights/resnet152.pdparams md5 checking...
2021-11-13 17:53:39,126 - INFO - Found /home/aistudio/.cache/paddle/hapi/weights/resnet152.pdparams
2021-11-13 17:53:41,081 - INFO - unique_endpoints {''}
2021-11-13 17:53:41,082 - INFO - File /home/aistudio/.cache/paddle/hapi/weights/resnet152.pdparams md5 checking...
2021-11-13 17:53:41,877 - INFO - Found /home/aistudio/.cache/paddle/hapi/weights/resnet152.pdparams


In [21]:
img_test_transforms = trans.Compose([
    trans.CropCenterSquare(),
    trans.Resize((image_size, image_size))
])

oct_test_transforms = trans.Compose([
    trans.CenterCrop([256] + oct_img_size)
])

test_dataset = GAMMA_sub1_dataset(dataset_root=testset_root, 
                        img_transforms=img_test_transforms,
                        oct_transforms=oct_test_transforms,
                        mode='test')

In [22]:
cache = []
for fundus_img, oct_img, idx in test_dataset:
    print('\r'+str(idx)+'/100',end='')
    fundus_img = fundus_img[np.newaxis, ...]
    oct_img = oct_img[np.newaxis, ...]

    fundus_img = paddle.to_tensor((fundus_img / 255.).astype("float32"))
    oct_img = paddle.to_tensor((oct_img / 255.).astype("float32"))

    logits = model(fundus_img, oct_img)
    cache.append([idx, logits.numpy().argmax(1)])

0142/100

Premature end of JPEG file


0176/100

In [23]:
submission_result = pd.DataFrame(cache, columns=['data', 'dense_pred'])

In [24]:
submission_result['non'] = submission_result['dense_pred'].apply(lambda x: int(x[0] == 0))
submission_result['early'] = submission_result['dense_pred'].apply(lambda x: int(x[0] == 1))
submission_result['mid_advanced'] = submission_result['dense_pred'].apply(lambda x: int(x[0] == 2))

In [25]:
submission_result[['data', 'non', 'early', 'mid_advanced']].to_csv("./submission_sub1.csv", index=False)

# 总结

- 比赛基线的双分支网络还是靠谱的，直接套用可以获得较高的得分。
   
- 在目前的网络结构下，我很难看到更好的结果了，可以考虑对网络结构进行变迁从而获得更好的结果。